# Semantic Segmentation Quiz

### Maintainer : Wonkwang Lee (wonkwang.lee@kaist.ac.kr)

In [12]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=(1, 224, 224, 3))
label = tf.broadcast_to(tf.expand_dims(tf.expand_dims(tf.one_hot(1, 21), -1), -1), (1, 21, 224, 224))

/home/com02/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# Quiz 1: Building toy FCN blocks

![FCN](http://deeplearning.net/tutorial/_images/cat_segmentation.png)


## Instruction:
Build toy FCN blocks using `models.Sequential()`, `layers.Conv2D`, `layers.Maxpooling2D`, `layer.UpSampling2D`. <br>
Please refer to the printed output for `fcn.summary()`

#### hint1 : https://www.tensorflow.org/beta/tutorials/images/intro_to_cnns
#### hint2 : https://keras.io/layers/convolutional/#conv2d
#### hint3 : https://keras.io/layers/pooling/#maxpooling2d
#### hint4 : https://keras.io/layers/convolutional/#upsampling2d

In [13]:
def FCN():
    model = models.Sequential()
    model.add(layers.Conv2D(96, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    ''' implement remaining modules here '''
    model.add(layers.MaxPooling2D((2, 2), strides=2))
    model.add(layers.Conv2D(256, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=2))
    model.add(layers.Conv2D(384, 3, activation='relu', padding='same'))
    model.add(layers.Conv2D(384, 3, activation='relu', padding='same'))
    model.add(layers.Conv2D(256, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=2))
    model.add(layers.Conv2D(4096, 3, activation='relu', padding='same'))
    model.add(layers.Conv2D(4096, 3, activation='relu', padding='same'))
    model.add(layers.Conv2D(21, 3, activation='relu', padding='same'))
    model.add(layers.UpSampling2D((8, 8)))
    
    return model

In [14]:
fcn = FCN()
fcn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 224, 224, 96)      2688      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 112, 112, 96)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 112, 112, 256)     221440    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 256)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 56, 56, 384)       885120    
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 56, 56, 384)       1327488   
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 56, 56, 256)      

In [15]:
print(x.shape)

(1, 224, 224, 3)


In [16]:
prediction = fcn(x)
prediction

<tf.Tensor 'sequential_1/up_sampling2d_1/ResizeNearestNeighbor:0' shape=(1, 224, 224, 21) dtype=float32>

---

# Quiz 2: Preprocess data to calculate cross-entropy loss


#### hint1 : https://www.tensorflow.org/api_docs/python/tf/losses/softmax_cross_entropy
#### hint2 : https://www.tensorflow.org/api_docs/python/tf/transpose

In [17]:
def preprocess_label(label):
    ''' write down proper pre-processing operation here'''
    label = tf.transpose(label, perm=(0, 2, 3, 1))
    return label

def calculate_loss(prediction, label):
    loss = tf.losses.softmax_cross_entropy(label, prediction)
    return loss

In [18]:
print('prediction shape :', prediction.shape)
print('label shape :', label.shape)

prediction shape : (1, 224, 224, 21)
label shape : (1, 21, 224, 224)


In [19]:
label = preprocess_label(label)
print('label shape :', label.shape)

label shape : (1, 224, 224, 21)


In [20]:
loss = calculate_loss(prediction, label)
loss

W0801 15:38:42.844196 140482937288512 deprecation.py:323] From /home/com02/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<tf.Tensor 'softmax_cross_entropy_loss/value:0' shape=() dtype=float32>

---

# Quiz 3: Building toy DeconvNet

![deconvnet](https://miro.medium.com/max/1400/1*LW8Anre45o9nfamxIVTY8Q.png)

## Instruction:
Build toy DeconvNet blocks using `models.Sequential()`, `layers.Conv2D`, `layers.Maxpooling2D`, `layer.Conv2DTranspose`. <br>
Please refer to the printed output for `deconvnet.summary()` <br>
<br>
Note: we use `layer.Conv2DTranspose` for upsampling feature maps instead of un-pooling operation

In [32]:
def DeconvNet():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    ''' implement remaining modules here '''
    model.add(layers.MaxPooling2D((2, 2), strides=2))
    model.add(layers.Conv2D(64, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=2))
    model.add(layers.Conv2D(128, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=2))
    model.add(layers.Conv2D(256, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=2))
    model.add(layers.Conv2D(512, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=2))
    model.add(layers.Conv2DTranspose(512, 3, strides=2, padding='same', activation='relu'))
    model.add(layers.Conv2DTranspose(256, 3, strides=2, padding='same', activation='relu'))
    model.add(layers.Conv2DTranspose(128, 3, strides=2, padding='same', activation='relu'))
    model.add(layers.Conv2DTranspose(64, 3, strides=2, padding='same', activation='relu'))
    model.add(layers.Conv2DTranspose(21, 3, strides=2, padding='same', activation='relu'))
    return model

In [34]:
deconvnet = DeconvNet()
deconvnet.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 28, 28, 256)      

In [35]:
prediction = deconvnet(x)
prediction

<tf.Tensor 'sequential_8/conv2d_transpose_14/Relu:0' shape=(1, 224, 224, 21) dtype=float32>

---

# Quiz 4 : Feature Pyramid

<img src="toy_fpn.png" width="600">

## Instruction
Implement `upsample_and_merge` function

#### hint1 : https://www.tensorflow.org/api_docs/python/tf/concat

In [36]:
conv_block1 = models.Sequential()
conv_block1.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
conv_block1.add(layers.MaxPool2D((2,2)))

conv_block2 = models.Sequential()
conv_block2.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
conv_block2.add(layers.MaxPool2D((2,2)))

In [37]:
out1 = conv_block1(x)
out1

<tf.Tensor 'sequential_9/max_pooling2d_41/MaxPool:0' shape=(1, 112, 112, 32) dtype=float32>

In [38]:
out2 = conv_block2(out1)
out2

<tf.Tensor 'sequential_10/max_pooling2d_42/MaxPool:0' shape=(1, 56, 56, 64) dtype=float32>

In [39]:
def upsample_and_merge(small, large):
    '''write down codes here'''
    upsample = layers.UpSampling2D(size=(2, 2))
    merged = layers.concatenate([large, upsample(small)], axis=3)
    return merged

In [40]:
upsample_and_merge(out2, out1)

<tf.Tensor 'concatenate/concat:0' shape=(1, 112, 112, 96) dtype=float32>

---

# Quiz 5 : Building blocks with atrous convolution

![fcn_atrous](fcn_atrous.png)
#### hint1 : https://keras.io/layers/convolutional/#conv2d

In [41]:
def Conv_Atrous():
    model = models.Sequential()
    ''' implement remaining modules here. note that the last four modules should be atrous convs'''
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(256, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(512, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    # add atrous convolution with rate 2
    model.add(layers.Conv2D(512, 3, dilation_rate=2, activation='relu', padding='same'))
    # add atrous convolution with rate 4
    model.add(layers.Conv2D(512, 3, dilation_rate=4, activation='relu', padding='same'))
    # add atrous convolution with rate 8
    model.add(layers.Conv2D(512, 3, dilation_rate=8, activation='relu', padding='same'))
    # add atrous convolution with rate 16
    model.add(layers.Conv2D(512, 3, dilation_rate=16, activation='relu', padding='same'))
    return model

In [42]:
conv_atrous = Conv_Atrous()
conv_atrous.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_53 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 28, 28, 256)     

In [19]:
conv_atrous = Conv_Atrous()
conv_atrous.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 28, 28, 256)       295168    
__________

---

# Quiz 6 : Building toy DeepLab

![aspp](aspp.png)

## Instruction
Complete ``DeepLab_Front`, conv_with_dilation_rate2`, `conv_with_dilation_rate4`, `conv_with_dilation_rate8`, `conv_with_dilation_rate16`, and `aspp`, where `aspp` is simply computed by concatenating all the `feature_map_rateX`

In [44]:
def DeepLab_Front():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    ''' implement remaining modules here '''
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(256, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(512, 3, activation='relu', padding='same'))
    return model

In [46]:
front = DeepLab_Front()
front.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_67 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 28, 28, 256)     

In [47]:
''' write down codes here '''
conv_with_dilation_rate2 = layers.Conv2D(512, 3, dilation_rate=2, activation='relu', padding='same')
conv_with_dilation_rate4 = layers.Conv2D(512, 3, dilation_rate=4, activation='relu', padding='same')
conv_with_dilation_rate8 = layers.Conv2D(512, 3, dilation_rate=8, activation='relu', padding='same')
conv_with_dilation_rate16 = layers.Conv2D(512, 3, dilation_rate=16, activation='relu', padding='same')

In [48]:
feature_map = front(x)
feature_map_rate2 = conv_with_dilation_rate2(feature_map)
feature_map_rate4 = conv_with_dilation_rate4(feature_map)
feature_map_rate8 = conv_with_dilation_rate8(feature_map)
feature_map_rate16 = conv_with_dilation_rate16(feature_map)

In [49]:
sppp = tf.concat([feature_map_rate2, feature_map_rate4, feature_map_rate8, feature_map_rate16], axis=3)
sppp

<tf.Tensor 'concat:0' shape=(1, 14, 14, 2048) dtype=float32>

---

# Quiz 7 : Building toy Unet

![unet](simple_unet.png)

## Instruction
implement a simple U-Net module using `ds`, `upsample`, and `deconv` module

In [50]:
def UNet_layer3(x):
    
    ds = layers.MaxPooling2D((2, 2))
    upsample = layers.UpSampling2D((2, 2))
    deconv = layers.Conv2DTranspose(3, (3, 3), activation='relu', strides=2, padding='same')
    
    out = ds(x)
    out_up = upsample(out)
    out_large = deconv(out)
    out = tf.concat([out_up, out_large], axis=3)
    return out

In [51]:
UNet_layer3(x)

<tf.Tensor 'concat_1:0' shape=(1, 224, 224, 6) dtype=float32>